In [74]:
import fitz
import re
from xml.etree.ElementTree import Element, SubElement, tostring
from xml.dom.minidom import parseString

pdf_doc = fitz.open('C:\\Users\\SNUH\\Desktop\\export\\14_74535622.pdf')
# number of pages
print(f"전체 Page 수: {pdf_doc.page_count}")
# Get the first page
page = pdf_doc.load_page(0)
# page 내의 텍스트 추출
print(page.get_text())

전체 Page 수: 18
Location: Unknown
HOLTER REPORT
PARK, HARAM
Patient Name:
74535622
ID:
17 yr
Age:
Female
Gender:
Date of Birth:
, 
Overreading Physician:
, 
Referring Physician:
, 
Ordering Physician:
Hook-Up Technician: , 
Indication/Diagnosis:
Medications:
Medications:
03-Aug-2011
Hookup Date:
10:56:00
Hookup Time:
23:44:00
Duration:
General
110363 QRS complexes
488 Ventricular beats (< 1%)
654 Supraventricular beats (< 1%)
< 1 % of total time classified as noise
Heart Rates
49 Minimum at 05:40:33 04-Aug
78 Average
170 Maximum at 10:34:04 04-Aug
13971 Beats in tachycardia (>100 bpm), 13% total
10938 Beats in bradycardia (<60 bpm), 10% total
1.52 Seconds Max R-R at 05:03:06 04-Aug
Ventriculars (V, F, E, I)
442 Isolated
15 Couplets
6 Bigeminal cycles
3 Runs totaling 16 beats
9 Beats longest run 122 bpm 19:30:46 03-Aug
9 Beats fastest run 122 bpm 19:30:46 03-Aug
Supraventriculars (S, J, A)
649 Isolated
1 Couplets
0 Bigeminal cycles
1 Runs totaling 3 beats
3 Beats longest run 147 bpm 17:25

In [75]:
import fitz  # PyMuPDF

pdf_doc = fitz.open('C:\\Users\\SNUH\\Desktop\\export\\14_74535622.pdf')

# Initialize an empty string to hold all the text
all_text = ""

# Iterate through each page in the document
for page_num in range(pdf_doc.page_count):
    # Load the current page
    page = pdf_doc.load_page(page_num)
    # Extract text from the current page and add it to the all_text string
    all_text += page.get_text()

# Now all_text contains text from all pages
print(all_text)

Location: Unknown
HOLTER REPORT
PARK, HARAM
Patient Name:
74535622
ID:
17 yr
Age:
Female
Gender:
Date of Birth:
, 
Overreading Physician:
, 
Referring Physician:
, 
Ordering Physician:
Hook-Up Technician: , 
Indication/Diagnosis:
Medications:
Medications:
03-Aug-2011
Hookup Date:
10:56:00
Hookup Time:
23:44:00
Duration:
General
110363 QRS complexes
488 Ventricular beats (< 1%)
654 Supraventricular beats (< 1%)
< 1 % of total time classified as noise
Heart Rates
49 Minimum at 05:40:33 04-Aug
78 Average
170 Maximum at 10:34:04 04-Aug
13971 Beats in tachycardia (>100 bpm), 13% total
10938 Beats in bradycardia (<60 bpm), 10% total
1.52 Seconds Max R-R at 05:03:06 04-Aug
Ventriculars (V, F, E, I)
442 Isolated
15 Couplets
6 Bigeminal cycles
3 Runs totaling 16 beats
9 Beats longest run 122 bpm 19:30:46 03-Aug
9 Beats fastest run 122 bpm 19:30:46 03-Aug
Supraventriculars (S, J, A)
649 Isolated
1 Couplets
0 Bigeminal cycles
1 Runs totaling 3 beats
3 Beats longest run 147 bpm 17:25:50 03-Aug
3 B

In [76]:
extracted_text=all_text

In [77]:
import re
from xml.etree.ElementTree import Element, SubElement, tostring
import xml.dom.minidom
import xml.etree.ElementTree as ET

# PDF text extraction
root = Element('HolterReport')

# Parsing the text
# patient_name = re.findall(r"HOLTER REPORT\n(.+)\nPatient Name:", extracted_text)[0]
patient_id = re.findall(r"Patient Name:\n(\d+)\nID:", extracted_text)[0]
hookup_date = re.findall(r"Medications:\n(\d+-\w+-\d+)\nHookup Date:", extracted_text)[0]
hookup_time = re.findall(r"Hookup Date:\n(\d+:\d+:\d+)\nHookup Time:", extracted_text)[0]
duration = re.findall(r"Hookup Time:\n(\d+:\d+:\d+)\nDuration:", extracted_text)[0]

# Parsing General section
general_section = re.search(r"General\n(.+?)Heart Rates", extracted_text, re.DOTALL).group(1)
qrs_complexes = re.search(r"(\d+) QRS complexes", general_section).group(1)
ventricular_beats = re.search(r"(\d+) Ventricular beats", general_section).group(1)
supraventricular_beats = re.search(r"(\d+) Supraventricular beats", general_section).group(1)
noise_percentage = re.search(r"(< 1 %) of total time classified as noise", general_section).group(1)


# Constructing the XML
patient_info = SubElement(root, 'PatientInfo')
#SubElement(patient_info, 'Name').text = patient_name
SubElement(patient_info, 'PID').text = patient_id
SubElement(patient_info, 'HookupDate').text = hookup_date
SubElement(patient_info, 'HookupTime').text = hookup_time
SubElement(patient_info, 'Duration').text = duration

# General
general = SubElement(root, 'General')
SubElement(general, 'QRScomplexes').text = qrs_complexes
SubElement(general, 'VentricularBeats').text = ventricular_beats
SubElement(general, 'SupraventricularBeats').text = supraventricular_beats
SubElement(general, 'NoisePercentage').text = noise_percentage

# Heart Rates section
heart_rates = SubElement(root, 'HeartRates')
patterns_hr = [
    (r"(\d+) Minimum at ([\d:]+ \d+-\w+)", 'MinimumRate', 'Timestamp'),
    (r"(\d+) Average", 'AverageRate', None),
    (r"(\d+) Maximum at ([\d:]+ \d+-\w+)", 'MaximumRate', 'Timestamp'),
    (r"(\d+) Beats in tachycardia \(>[0-9]+ bpm\), (\d+)% total", 'TachycardiaBeats', 'TachycardiaPercentage'),
    (r"(\d+) Beats in bradycardia \(<60 bpm\), (\d+)% total", 'BradycardiaBeats', 'BradycardiaPercentage'),
]

for pattern, main_tag, sub_tag in patterns_hr:
    match = re.search(pattern, extracted_text)
    if match:
        if sub_tag:
            element = SubElement(heart_rates, main_tag)
            SubElement(element, sub_tag).text = match.group(2)
            element.text = match.group(1)  # Main value
        else:
            SubElement(heart_rates, main_tag).text = match.group(1)
            
max_rr_match = re.search(r"(\d+\.\d+) Seconds Max R-R at ([\d:]+ \d+-\w+)", extracted_text)
if max_rr_match:
    max_rr = SubElement(heart_rates, 'SecondsMaxRR')
    SubElement(max_rr, 'Seconds').text = max_rr_match.group(1)
    SubElement(max_rr, 'Timestamp').text = max_rr_match.group(2)

                    
# Ventriculars section extraction
ventriculars_match = re.search(r"Ventriculars \(V, F, E, I\)\n([\s\S]+?)\nSupraventriculars \(S, J, A\)", extracted_text)
if ventriculars_match:
    ventriculars_section = ventriculars_match.group(1)
else:
    ventriculars_section = ""

# Supraventriculars section extraction
supraventriculars_match = re.search(r"Supraventriculars \(S, J, A\)\n([\s\S]+?)Interpretation", extracted_text)
if supraventriculars_match:
    supraventriculars_section = supraventriculars_match.group(1)
else:
    supraventriculars_section = ""

# Regex patterns for Ventriculars and Supraventriculars
ventriculars_patterns = [
    (r"(\d+) Isolated", ['Isolated']),
    (r"(\d+) Couplets", ['Couplets']),
    (r"(\d+) Bigeminal cycles", ['BigeminalCycles']),
    (r"(\d+) Runs totaling (\d+) beats", ['Runs', ('RunsDetails', 'TotalBeats')]),
    (r"(\d+) Beats longest run (\d+) bpm ([\d:]+ \d+-\w+)", [('LongestRun', 'Beats'), ('LongestRun', 'BPM'), ('LongestRun', 'Timestamp')]),
    (r"(\d+) Beats fastest run (\d+) bpm ([\d:]+ \d+-\w+)", [('FastestRun', 'Beats'), ('FastestRun', 'BPM'), ('FastestRun', 'Timestamp')])
]

supraventriculars_patterns = [
    (r"(\d+) Isolated", ['Isolated']),
    (r"(\d+) Couplets", ['Couplets']),
    (r"(\d+) Bigeminal cycles", ['BigeminalCycles']),
    (r"(\d+) Runs totaling (\d+) beats", ['Runs', ('RunsDetails', 'TotalBeats')]),
    (r"(\d+) Beats longest run (\d+) bpm ([\d:]+ \d+-\w+)", [('LongestRun', 'Beats'), ('LongestRun', 'BPM'), ('LongestRun', 'Timestamp')]),
    (r"(\d+) Beats fastest run (\d+) bpm ([\d:]+ \d+-\w+)", [('FastestRun', 'Beats'), ('FastestRun', 'BPM'), ('FastestRun', 'Timestamp')])
]

# Ventriculars section to add xml
ventriculars_xml = ET.SubElement(root, "Ventriculars")
for pattern, tags in ventriculars_patterns:
    match = re.search(pattern, ventriculars_section)
    if match:
        for tag_index, tag in enumerate(tags):
            if isinstance(tag, tuple):  # 복잡한 태그 처리
                parent_tag = ET.SubElement(ventriculars_xml, tag[0])
                ET.SubElement(parent_tag, tag[1]).text = match.group(tag_index + 1)
            else:
                ET.SubElement(ventriculars_xml, tag).text = match.group(tag_index + 1)

# Supraventriculars section to add xml
supraventriculars_xml = ET.SubElement(root, "Supraventriculars")
for pattern, tags in supraventriculars_patterns:
    match = re.search(pattern, supraventriculars_section)
    if match:
        for tag_index, tag in enumerate(tags):
            if isinstance(tag, tuple):  # 복잡한 태그 처리
                parent_tag = ET.SubElement(supraventriculars_xml, tag[0])
                ET.SubElement(parent_tag, tag[1]).text = match.group(tag_index + 1)
            else:
                ET.SubElement(supraventriculars_xml, tag).text = match.group(tag_index + 1)

xml_str = ET.tostring(root, 'utf-8')
parsed_str = xml.dom.minidom.parseString(xml_str)
pretty_xml_str = parsed_str.toprettyxml(indent="   ")

xml_file_path = 'C:\\Users\\SNUH\\Desktop\\export\\HolterReport.xml'
with open(xml_file_path, "w") as xml_file:
    xml_file.write(pretty_xml_str)

In [82]:
print(pretty_xml_str)

<?xml version="1.0" ?>
<HolterReport>
   <PatientInfo>
      <PID>74535622</PID>
      <HookupDate>03-Aug-2011</HookupDate>
      <HookupTime>10:56:00</HookupTime>
      <Duration>23:44:00</Duration>
   </PatientInfo>
   <General>
      <QRScomplexes>110363</QRScomplexes>
      <VentricularBeats>488</VentricularBeats>
      <SupraventricularBeats>654</SupraventricularBeats>
      <NoisePercentage>&lt; 1 %</NoisePercentage>
   </General>
   <HeartRates>
      <MinimumRate>
         49
         <Timestamp>05:40:33 04-Aug</Timestamp>
      </MinimumRate>
      <AverageRate>78</AverageRate>
      <MaximumRate>
         170
         <Timestamp>10:34:04 04-Aug</Timestamp>
      </MaximumRate>
      <TachycardiaBeats>
         13971
         <TachycardiaPercentage>13</TachycardiaPercentage>
      </TachycardiaBeats>
      <BradycardiaBeats>
         10938
         <BradycardiaPercentage>10</BradycardiaPercentage>
      </BradycardiaBeats>
      <SecondsMaxRR>
         <Seconds>1.52</Seconds>
